In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as utils

In [2]:
num_hospitals = 10 #number of hospitals providing their models
num_images = 10000 #number of un-labelled images that we want to train
num_labels = 10 #number of mutually exclusive lables that the iamges can have

In [3]:
#generate array of fake predicted labels for each hospital for all our images with shape num_images * num_hospitals
predicted_labels = (np.random.rand(num_hospitals, num_images) * num_labels).astype(int).transpose()

In [4]:
#check our predicted labels array
print(predicted_labels.shape) #shape of the predicted lables for each hospital for all of our training images
print(predicted_labels[0]) #one image predicted labels

(10000, 10)
[8 8 5 7 8 2 3 3 1 9]


In [5]:
#for each image generate a label by getting the most agreed upon prediction by all the hospital adding laplacian noise to
#the count of each occurrence of a prediction label 
private_combined_labels = [] 
for image in predicted_labels:
    
    label_count = np.bincount(image, minlength=num_labels)
    
    #decide upon the epsilon and beta values
    epsilon = 0.1
    beta = 1/epsilon
    
    #add laplacian noise to each count
    for i in range(len(label_count)):
        label_count[i] += np.random.laplace(0, beta, 1)
    
    chosen_label = np.argmax(label_count)
    private_combined_labels.append(chosen_label)

In [6]:
#convert the private labels to tensor
private_combined_labels = torch.FloatTensor(private_combined_labels).type(torch.LongTensor)
private_combined_labels.shape

torch.Size([10000])

In [7]:
#generate fake image data to train a neural netwrok on the private generated labels
images = torch.randn((num_images, 1, 28, 28))
images.shape

torch.Size([10000, 1, 28, 28])

In [8]:
train_dataset = utils.TensorDataset(images, private_combined_labels)
train_dataloader = utils.DataLoader(train_dataset, batch_size=64)

In [9]:
#Define our network architecture here
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_labels)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [10]:
# Create the network, define the criterion and optimizer
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [11]:
# Train the network here
epochs = 10

for e in range(epochs):
    running_loss = 0
    for images, labels in train_dataloader:        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_dataloader)}")

Training loss: 2.305204453741669
Training loss: 2.2787237288845574
Training loss: 2.1791197790461743
Training loss: 2.073322765386788
Training loss: 1.9300715224758076
Training loss: 1.8028447650800086
Training loss: 1.6058019469877718
Training loss: 1.3859849853120791
Training loss: 1.2127848998376518
Training loss: 1.0192063534335725
